In [ ]:
#librerias
import pandas as pd
import json

In [33]:
"""
Aplicamos un etl a los datos descargados de la página de la salud de NYen formato json y los pasamos a formato CSV
"""

# Rutas de los archivos JSON y CSV
ruta_json = r'H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\raw\departamento salubridad NY\rows.json'
ruta_csv = r'H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\raw\departamento salubridad NY\rows.csv'

with open(ruta_json, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Detectamos la estructura del JSON
if isinstance(data, dict) and 'data' in data:
    df = pd.DataFrame(data['data'])
elif isinstance(data, list):
    df = pd.DataFrame(data)
else:
    raise ValueError("❌ Estructura JSON no reconocida")

# Guardamos el DataFrame en un CSV sin cabecera
df.to_csv(ruta_csv, index=False, header=False, encoding='utf-8-sig')
print(f"✅ CSV generado desde JSON en: {ruta_csv}")

# Añadimos las cabeceras al archivo CSV
columnas_originales = [
    "row_id", "uuid", "col_1", "timestamp_1", "col_2", "timestamp_2", "col_3", "json_data",
    "borough", "restaurant_name", "location_type", "owner", "address", "inspection_id",
    "col_4", "compliance_status", "col_5", "inspection_date", "agency", "zip_code",
    "latitude", "longitude", "grade", "violation_points", "violation_code", 
    "bin", "bbl", "neighborhood"
]

df = pd.read_csv(ruta_csv, header=None, names=columnas_originales, low_memory=False, dtype=str)
df.to_csv(ruta_csv, index=False, encoding='utf-8-sig')
print(f"✅ Cabeceras aplicadas a: {ruta_csv}")

# Filtrarmos las columnas relevantes a usar
columnas_compatibles = {
    "restaurant_name": "nombre",
    "address": "direccion",
    "borough": "ciudad",
    "zip_code": "codigo_postal",
    "latitude": "latitud",
    "longitude": "longitud",
    "compliance_status": "estado_cumplimiento",
    "inspection_date": "fecha_inspeccion",
    "violation_points": "puntos_violacion",
    "violation_code": "codigo_violacion"
}

df_final = df[list(columnas_compatibles.keys())].rename(columns=columnas_compatibles)

# Normalizamos todos los datos string a minúsculas (sin tocar headers)
for col in df_final.columns:
    df_final[col] = df_final[col].astype(str).str.lower().str.strip()

# Cargamos el archivo de dimensión de nombres de restaurantes
ruta_dim = r'H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\dim\dim_nombre_restaurante.csv'
dim_nombres = pd.read_csv(ruta_dim)

# Normalizamos 'nombre' en dimensión para que coincida
dim_nombres['nombre'] = dim_nombres['nombre'].astype(str).str.lower().str.strip()

# Eliminamos valores nulos y duplicados
df_final = df_final[df_final['nombre'].notna() & (df_final['nombre'].str.strip() != '')]
df_final = df_final.merge(dim_nombres, on='nombre', how='left')

# Ordenamos las columnas
orden_final = [
    'id_nombre', 'nombre', 'direccion', 'ciudad', 'codigo_postal',
    'latitud', 'longitud', 'estado_cumplimiento',
    'fecha_inspeccion', 'puntos_violacion', 'codigo_violacion'
]
df_final = df_final[orden_final]

# Guardamos el dataset limpio en un archivo CSV
ruta_salida = r'H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\salubridad_merge_yelp.csv'
df_final.to_csv(ruta_salida, index=False, encoding='utf-8-sig')

print(f"✅ Dataset limpio, normalizado en minúsculas y unido guardado en:\n{ruta_salida}")



✅ CSV generado desde JSON en: H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\raw\departamento salubridad NY\rows.csv
✅ Cabeceras aplicadas a: H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\raw\departamento salubridad NY\rows.csv
✅ Dataset limpio, normalizado en minúsculas y unido guardado en:
H:\git\proyecto grupal 2\Yelp-Gmaps-Proyecto-DS\data\processed\salubridad_merge_yelp.csv
